Here’s Python code for a backtesting framework that simulates a new minute-bar every minute using historical data. This framework processes the data incrementally, simulating how a live trading system would handle incoming minute bars. It uses a simple moving average crossover strategy as an example.

In [ ]:
import pandas as pd
import time
from datetime import datetime, timedelta

# Simulate historical data streaming minute-by-minute
def simulate_minutebar_stream(data, strategy, initial_balance=10000):
    """
    Simulate minute-by-minute data streaming and backtest a trading strategy.

    Parameters:
        data (pd.DataFrame): Historical minute-bar data.
        strategy (function): Trading strategy function that returns signals.
        initial_balance (float): Starting capital.
    """
    # Portfolio variables
    cash = initial_balance
    position = 0
    portfolio_value = initial_balance

    # Track performance
    performance = []

    print("Starting backtest...\n")

    for i in range(len(data)):
        # Simulate streaming of the next bar
        current_bar = data.iloc[i]
        current_time = current_bar["timestamp"]
        current_price = current_bar["close"]

        # Generate signal based on the strategy
        signal = strategy(data.iloc[:i + 1])

        # Execute trades based on the signal
        if signal == "buy" and cash >= current_price:
            position += 1
            cash -= current_price
            print(f"[{current_time}] BUY at {current_price:.2f}")
        elif signal == "sell" and position > 0:
            position -= 1
            cash += current_price
            print(f"[{current_time}] SELL at {current_price:.2f}")

        # Calculate portfolio value
        portfolio_value = cash + position * current_price
        performance.append({"timestamp": current_time, "portfolio_value": portfolio_value})

        # Simulate real-time delay (optional)
        time.sleep(0.1)  # Adjust for a faster or slower simulation

    # Convert performance to DataFrame for analysis
    performance_df = pd.DataFrame(performance)
    print("\nBacktest complete.")
    return performance_df

# Simple Moving Average Crossover Strategy
def moving_average_crossover_strategy(data, short_window=5, long_window=10):
    """
    A simple moving average crossover strategy.

    Parameters:
        data (pd.DataFrame): Minute-bar data up to the current point.
        short_window (int): Short moving average window.
        long_window (int): Long moving average window.

    Returns:
        str: "buy", "sell", or "hold".
    """
    if len(data) < long_window:
        return "hold"  # Not enough data for the strategy

    data["short_ma"] = data["close"].rolling(window=short_window).mean()
    data["long_ma"] = data["close"].rolling(window=long_window).mean()

    if data["short_ma"].iloc[-1] > data["long_ma"].iloc[-1]:
        return "buy"
    elif data["short_ma"].iloc[-1] < data["long_ma"].iloc[-1]:
        return "sell"
    else:
        return "hold"

# Load historical minute-bar data
def load_data():
    """
    Load historical minute-bar data for testing.
    Replace this with your own data loading logic.

    Returns:
        pd.DataFrame: Historical minute-bar data.
    """
    # Example data (replace this with your actual data)
    data = {
        "timestamp": [datetime(2023, 1, 1, 9, 30) + timedelta(minutes=i) for i in range(120)],
        "open": [100 + i * 0.1 for i in range(120)],
        "high": [101 + i * 0.1 for i in range(120)],
        "low": [99 + i * 0.1 for i in range(120)],
        "close": [100 + i * 0.1 for i in range(120)],
        "volume": [1000 + i * 10 for i in range(120)],
    }
    return pd.DataFrame(data)

# Main function
if __name__ == "__main__":
    # Load the data
    historical_data = load_data()

    # Simulate the backtest
    performance = simulate_minutebar_stream(
        historical_data, 
        moving_average_crossover_strategy,
        initial_balance=10000
    )

    # Analyze the results
    print("\nPerformance Summary:")
    print(performance)


Explanation of Code
Minute-bar Simulation:

The simulate_minutebar_stream function processes the data row by row, simulating real-time minute-bar data updates.
It delays processing using time.sleep() to emulate live conditions (can be adjusted or removed for testing).
Trading Strategy:

The moving_average_crossover_strategy calculates short-term and long-term moving averages and generates "buy", "sell", or "hold" signals.
Portfolio Management:

Tracks cash, positions, and portfolio value as trades are executed.
Ensures there’s sufficient cash for buying and existing positions for selling.
Data Requirements:

Replace the load_data function with your own historical data source (e.g., CSV or database).
Performance Tracking:

Logs portfolio value over time for analysis.
Returns a DataFrame for further analysis and visualization.
Next Steps
Replace the dummy dataset with your actual historical minute-bar data.
Add transaction cost modeling (e.g., commissions and slippage).
Visualize results with matplotlib or plotly.
Optimize strategy parameters using historical testing.